In [ ]:
# load libraries and scripts
import JSON
using Plots
using HDF5
using Trapz
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")

In [ ]:
#load borders
border = import_border("../data/border.json")

# create the lattice grid
dx = 20
Nx, Ny, xrange, yrange, isinside, isborder, n = get_grid(border, Float64(dx))

# define x and y coordinate for plotting
x = repeat(reshape(xrange,1,Nx), Ny, 1)
y = repeat(reshape(yrange,Ny,1), 1, Nx);

In [ ]:
# compute or load grid parameters
recompute_param = true
sigma = 50.
if(recompute_param || isfile("../numerics/grid_params_" * string(dx) * ".h5"))
    bx, by, p, m, d = get_params(isinside, isborder, sigma, Float64(dx), yrange, xrange, "../data/pantagruel.h5",
        "../numerics/grid_params_" * string(dx) * ".h5")
else
    bx, by, p, m, d = get_params(isinside, "../numerics/grid_params_" * string(dx) * ".h5")
end

In [ ]:
trapz((yrange,xrange), m)

In [ ]:
# plot some physical quantities
plot(do_plot(isinside, p), do_plot(isinside, m), do_plot(isinside, d), layout=(3,1), size=(500,900))


In [ ]:
println(minimum(m[isinside]))
println(minimum(d[isinside]))

In [ ]:
# compute, iteratively, the stable solution
interval = 1000
Niter = 40000
th = zeros(Ny, Nx)

@time begin
    for k in 1:Niter
        if(mod(k,interval) == 0)
            temp = copy(th)
        end
        Threads.@threads for i in 2:Ny-1
            Threads.@threads for j in 2:Nx-1
                if(isinside[i,j])
                    bij = (by[i-1,j] + by[i,j] + bx[i,j] + bx[i,j+1])
                    th[i,j] = (by[i,j] * th[i+1,j] + by[i-1,j] * th[i-1,j] + 
                        bx[i,j+1] * th[i,j+1] + bx[i,j] * th[i,j-1] + dx^2*p[i,j]) / bij
                end
            end
        end
        Threads.@threads for k in 1:size(n,1)
            i = Int64(n[k,1])
            j = Int64(n[k,2])
            nx = n[k,4]
            ny = n[k,3]
            if(nx == 1)
                th[i,j] = th[i,j-2]
            elseif(nx == -1)
                th[i,j] = th[i,j+2]
            end
            if(ny == 1)
                th[i,j] = th[i-2,j]
            elseif(ny == -1)
                th[i,j] = th[i+2,j]
            end
        end  
        if(mod(k,interval) == 0)
            println([k maximum(abs.(th-temp))])
        end

    end
end


In [ ]:
# plot the result
do_plot(isinside, th)

In [ ]:
# save the result
fid = h5open("../numerics/stable_" * string(dx) * ".h5", "w")
write(fid, "th", th)
close(fid)